In [33]:
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from matplotlib import pyplot
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import ExtraTreesRegressor
from imblearn.over_sampling import ADASYN
from matplotlib import pyplot
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, explained_variance_score, max_error, mean_absolute_error, mean_squared_error, median_absolute_error

In [34]:
df = pd.read_csv('/home/junio/Desktop/Ju/mywaytohealth2020/jupyter/data/01.PATIENTS_VISITES-no_control_no_spurious_data-median-imputation_ADASYN.csv',sep=',')

In [35]:
df.shape

(1042, 24)

**100 rounds random sampling validation**

In [28]:
#The problem is imbalanced, the averageing MUST be 'macro' to reflect the minority class missclassifications
if True:
    selected_fields = ['age', 'egfdrs001', 'nycturie_nb', 'score_depression', 'imc', 'perimetre_cervical', 'tour_de_hanches', 'padiast']    
    n_folds = 100
    df = pd.read_csv('/home/junio/Desktop/Ju/mywaytohealth2020/jupyter/data/01.PATIENTS_VISITES-no_control_no_spurious_data-median-imputation.csv',sep=',')
    comb_to_scores_MAP = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    print(selected_fields)
    for fold in range(n_folds):
        df = sklearn.utils.shuffle(df)
        x = df[selected_fields+['polysomnographie_iah']]
        y_class = df[['iah_class']]
        x, y = ADASYN().fit_resample(x, y_class)
        x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y.values.ravel(),test_size=0.33)

        y_train = x_train[['polysomnographie_iah']]
        x_train = x_train.drop(columns=['polysomnographie_iah'])
        y_test = x_test[['polysomnographie_iah']]
        x_test = x_test.drop(columns=['polysomnographie_iah'])

        gnb = ExtraTreesRegressor(n_estimators=150)
        gnb.fit(x_train, y_train.values.ravel())
        y_pred = gnb.predict(x_test)

        comb_to_scores_MAP[0] += r2_score(y_test, y_pred)
        comb_to_scores_MAP[1] += explained_variance_score(y_test, y_pred)
        comb_to_scores_MAP[2] += mean_absolute_error(y_test, y_pred)
        comb_to_scores_MAP[3] += mean_squared_error(y_test, y_pred)
        comb_to_scores_MAP[4] += median_absolute_error(y_test, y_pred)
        
    comb_to_scores_MAP[0] = comb_to_scores_MAP[0] / n_folds
    comb_to_scores_MAP[1] = comb_to_scores_MAP[1] / n_folds
    comb_to_scores_MAP[2] = comb_to_scores_MAP[2] / n_folds
    comb_to_scores_MAP[3] = comb_to_scores_MAP[3] / n_folds
    comb_to_scores_MAP[4] = comb_to_scores_MAP[4] / n_folds

    print('R^2 score',comb_to_scores_MAP[0])
    print('Explained variance',comb_to_scores_MAP[1])
    print('Mean absolute error',comb_to_scores_MAP[2])
    print('Mean squared error',comb_to_scores_MAP[3])
    print('Root mean squared error',pow(comb_to_scores_MAP[3],0.5))
    print('Median absolute error',comb_to_scores_MAP[4])

['age', 'egfdrs001', 'nycturie_nb', 'score_depression', 'imc', 'perimetre_cervical', 'tour_de_hanches', 'padiast']
R^2 score 0.412733766426485
Explained variance 0.41388782111773004
Mean absolute error 9.599359132938803
Mean squared error 264.22471135948837
Median absolute error 4.1538048827342715


In [30]:
print('Root mean squared error',pow(comb_to_scores_MAP[3],0.5))

Root mean squared error 16.25499035248832
